In [189]:
import pandas as pd
import numpy as np
horizon_path = '/home/dparks/Downloads/rmotc/DataSets/Seismic/CD files/3D_Seismic/3DHorizons.xyz'

#from the sismic
in_max = 345
xl_max = 188

in_min = 1
xl_min = 1

sample_rate = 0.002

in_len = (in_max - in_min) + 1;
xl_len = (xl_max - xl_min) + 1;

def hznToNpInterp(hzn):
    from scipy.interpolate import griddata
    points = []
    values = []
    
    grid_x, grid_y = np.mgrid[0:in_len:in_len,0:xl_len,xl_len]
    
    for i,row in hzn.iterrows():
        x = int(row['inline'] - in_min)
        y = int(row['xline'] - xl_min)

        points.append((x,y))
        values.append(float(row['Time']))
        
    result = griddata(points, values, (grid_x, grid_y), method='cubic') 
    print(result)
    return result

def hznToNp(hzn):
    result = np.zeros((in_len, xl_len))
    
    for i,row in hzn.iterrows():
        x = int(row['inline'] - in_min)
        y = int(row['xline'] - xl_min)

        result[x][y] = float(row['Time'])
        
    return result

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def read_horizons(horizon_path):
    with open(horizon_path) as f:
        hznDF = pd.read_csv(f, header=0, delim_whitespace=True)

        hznDF['inline'].astype(np.int32)
        hznDF['xline'].astype(np.int32)
        hznDF['Time'].astype(np.float32)

        gb = hznDF.groupby("Horizon")

        horizons = gb.apply(hznToNp)
        return list(zip(gb.groups.keys(), horizons)) 
   
horizons = read_horizons(horizon_path)

In [190]:
def show_plot(i):
    name, horizon = horizons[i]
    flattened = horizon.flatten() 

    print(name)
    nonZeroMin = min(filter(lambda x: x != 0, flattened.tolist()))
    
    plt.figure(figsize=(16,8))
    plt.imshow(horizon, cmap="jet", vmin=nonZeroMin, vmax=horizon.max(), aspect=0.25)
    plt.colorbar(shrink=0.5)
    plt.show()

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
interact(show_plot, i=widgets.IntSlider(min=0, max=len(horizons)-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='i', max=8), Output()), _dom_classes=('widget-interact',)…

<function __main__.show_plot(i)>

In [26]:
filename = '/home/dparks/Downloads/rmotc/DataSets/Seismic/CD files/3D_Seismic/filt_mig.sgy'


from obspy.io.segy.segy import _read_segy
stream = _read_segy(filename, headonly=True)

In [21]:
sampleRate = stream.binary_file_header.sample_interval_in_microseconds
print(sampleRate)

2000
